In [1]:
# imports
import requests
import json

import open3d as o3d
import numpy as np

import subprocess

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:

# get the id of the first annotation
url = "http://localhost:4321/api/queries/findAnnotation"
response = requests.get(url)
id = json.loads(response.text)[0]
print(id)

4c2224dd-96d8-4921-824b-f82f2c1e3812


In [31]:
# Helper functions
def applyTrafoToSurface(port, surface_id, transformation_matrix) -> None:
    """
    Applies a transformation to a surface.
    """
    url = f"http://localhost:{port}/api/surfaces/{surface_id}/transform"

    payload = {
        "forward": transformation_matrix.tolist()
    }

    headers = {
        "Content-Type": "application/json"
    }

    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status() 
    print(response.text)    

def writePcdObj (outputFile, pcd) -> None:
    """
    Writes a point cloud to a .pcd and .obj file.
    """
    with open(outputFile, "w") as f:
        for point in pcd.points:
            f.write(f"v {point[0]} {point[1]} {point[2]}\n")

def openFileInMeshlab (file) -> None:
    """
    Opens a file in Meshlab.
    """
    # open cutout in MeshLab
    try:
        subprocess.Popen(["C:/Program Files/vcg/MeshLab/meshlab.exe", file])
        print("MeshLab started successfully.")
    except FileNotFoundError:
        print("MeshLab executable not found. Please ensure it is installed and in your system PATH.")

def load_obj_as_pointcloud(file_path):
    points = []
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith("v "):  # Vertex line
                parts = line.strip().split()
                x, y, z = map(float, parts[1:4])  # Extract X, Y, Z
                points.append([x, y, z])

    # Create Open3D PointCloud
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(np.array(points))
    return point_cloud


In [4]:
tgt_points_id = "3d87d707-5735-4d7d-a401-679ae2a748b5"
src_points_id = "4b854809-1601-4e7e-aa78-85aa19fa43d3"

In [6]:
# Define the URL and query parameters
url = "http://localhost:4322/api/annotations/3d87d707-5735-4d7d-a401-679ae2a748b5/points"

# Send a GET request to the Suave endpoint
response = requests.get(url)
tgt_points = np.array(json.loads(response.text))

print(f"tgt points: {tgt_points}")

# Define the URL and query parameters
url = "http://localhost:4321/api/annotations/4b854809-1601-4e7e-aa78-85aa19fa43d3/points"

# Send a GET request to the Suave endpoint
response = requests.get(url)
src_points = np.array(json.loads(response.text))

print(f"src points: {src_points}")


tgt points: [[137.15062074 -56.11408989 -29.09428223]
 [108.25235213 -46.44734016 -27.29873521]
 [ 72.28858124 -58.32665047 -23.85722113]
 [  4.47920139 -91.81704426 -42.33971145]]
src points: [[ 701757.85653848 3142014.52990013 1073527.84141308]
 [ 701746.07743941 3142024.38444733 1073498.90227216]
 [ 701763.76885274 3142030.26883148 1073463.93951211]
 [ 701813.23370933 3142061.36943788 1073394.22420414]]


In [30]:
# Convert numpy arrays to Open3D PointClouds
source_pcd = o3d.geometry.PointCloud()
source_pcd.points = o3d.utility.Vector3dVector(src_points)

target_pcd = o3d.geometry.PointCloud()
target_pcd.points = o3d.utility.Vector3dVector(tgt_points)

# Define correspondences as index pairs
correspondences = np.array([[0, 0], [1, 1], [2, 2], [3, 3]])
correspondences_o3d = o3d.utility.Vector2iVector(correspondences)

# Create a TransformationEstimationPointToPoint object
estimation = o3d.pipelines.registration.TransformationEstimationPointToPoint()

# Compute the rigid transformation matrix
transformation_matrix = estimation.compute_transformation(source_pcd, target_pcd, correspondences_o3d)

# Print the transformation matrix
print("Computed Transformation Matrix:")
print(transformation_matrix)

Computed Transformation Matrix:
[[-9.55238971e-02 -3.07990171e-01  9.46581872e-01  1.87031379e+04]
 [-9.57203477e-01  2.89405582e-01 -2.43162464e-03 -2.35037772e+05]
 [-2.73197160e-01 -9.06303737e-01 -3.22454412e-01  3.38547244e+06]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [10]:
# Apply transformation to target surface
tgt_surface_id = "191cef90-bcbc-4acb-9318-ae11aa6355d4"
src_surface_id = "cfd09c0d-b202-47cd-9e4c-bc66b8e7b275"
applyTrafoToSurface(4321, tgt_surface_id, transformation_matrix)


Received payload [[-0.09552389707154675, -0.30799017131390394, 0.9465818715052053, 18703.137882233565], [-0.957203477112682, 0.28940558150288864, -0.002431624635176946, -235037.77169871362], [-0.27319716047509696, -0.9063037370379798, -0.3224544118776125, 3385472.4427601513], [0, 0, 0, 1]]


In [13]:
cut_annotation_id = "4c2224dd-96d8-4921-824b-f82f2c1e3812"
url = "http://localhost:4321/api/queries/queryAnnotationAsObj"

# retrieve the cutout geometry
payload = {
    "annotationId": cut_annotation_id, 
    "queryAttributes":[], 
    "distanceToPlane": 100000.0,
    "outputReferenceFrame": "global",
    "outputGeometryType": "pointcloud"
}
print(payload)
response = requests.post(url, json=payload)
response.raise_for_status() 
filename = "./coutout.obj"
file = open(filename, "w")
file.write(response.text)
file.close()

{'annotationId': '4c2224dd-96d8-4921-824b-f82f2c1e3812', 'queryAttributes': [], 'distanceToPlane': 100000.0, 'outputReferenceFrame': 'global', 'outputGeometryType': 'pointcloud'}


In [25]:
tgt_filePath = "G:\PRo3D\AI-Mars-3D\Co-Registration\packageGP\m2020-zcam-kodiak\source\model_4b_kodiak_100m_zcam_scam.obj"
tgt_filePath_trans = "G:\PRo3D\AI-Mars-3D\Co-Registration\packageGP\m2020-zcam-kodiak\source\model_4b_kodiak_100m_zcam_scam_IAU.obj"

mesh = o3d.io.read_triangle_mesh(tgt_filePath)

mesh.vertices = o3d.utility.Vector3dVector(np.asarray(mesh.vertices, dtype=np.float64))

mesh = mesh.transform(transformation_matrix)
o3d.io.write_triangle_mesh(tgt_filePath_trans, mesh)

[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D WARNING] Write PNG failed: image has no data.
[Open3D WARNING] Write OBJ successful, but failed to write texture file.


True

PointCloud with 566250 points.

In [ ]:
# ICP fine registration

# Load the SRC and TGT meshes
src_pcd = load_obj_as_pointcloud(filename)
src_pcd.transform(transformation_matrix)

tgt_mesh = o3d.io.read_triangle_mesh("G:\PRo3D\AI-Mars-3D\Co-Registration\packageGP\m2020-zcam-kodiak\source\model_4b_kodiak_100m_zcam_scam.obj")

# Convert meshes to point clouds for registration
tgt_pcd = tgt_mesh.sample_points_uniformly(number_of_points=1000000)
tgt_pcd = tgt_pcd.voxel_down_sample(voxel_size=0.01)
src_pcd = src_pcd.voxel_down_sample(voxel_size=0.01)

#o3d.visualization.draw_geometries([tgt_pcd, src_pcd])

# Compute normals for both point clouds
# src_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
# tgt_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Define the pre-alignment (Helmert transformation)
# pre_alignment = transformation_matrix

# Apply the pre-alignment transformation to the TGT point cloud
# tgt_pcd.transform(pre_alignment)

# writePcdObj("src_pcd.obj", src_pcd)
# writePcdObj("tgt_pcd.obj", tgt_pcd)

# openFileInMeshlab("src_pcd.obj")
# openFileInMeshlab("tgt_pcd.obj")

# Perform fine registration using ICP
icp_result = o3d.pipelines.registration.registration_icp(
    tgt_pcd,
    src_pcd,
    max_correspondence_distance=1000.0,
    init=np.eye(4),
    estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(),
    criteria=o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=200)
)

# # Print the transformation matrices
# print("Pre-Alignment (Helmert) Transformation:")
# print(pre_alignment)

print("\nICP Refinement Transformation:")
print(icp_result.transformation)

[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8
[Open3D INFO] Skipping non-triangle primitive geometry of type: 8

ICP Refinement Transformation:
[[ 0.99866711  0.04741196  0.02039853  7.45731967]
 [-0.0454661   0.99516026 -0.08711423  7.20467192]
 [-0.02443006  0.08607068  0.99598946  8.01979755]
 [ 0.          0.          0.          1.        ]]


MeshLab started successfully.


In [ ]:
# Apply final trafo to target surface
tgt_surface_id = "191cef90-bcbc-4acb-9318-ae11aa6355d4"

url = f"http://localhost:4321/api/surfaces/{tgt_surface_id}/transform"

payload = {
    "forward": icp_result.transformation.tolist()
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)
response.raise_for_status() 
print(response.text)

In [ ]:
# GREETING TEST

# Define the URL and query parameters
url = "http://localhost:4322/api/exp/greet/Orti"
url = "http://localhost:4322/api/exp/greet"
params = {"name": "Orti"}

# Send a GET request to the Suave endpoint
response = requests.get(url, params=params)

# Check the response
if response.status_code == 200:
    print(f"Response: {response.text}")
else:
    print(f"Error: {response.status_code}, Message: {response.text}")